In [1]:
import duckdb

In [6]:
con = duckdb.connect(database=":memory", read_only=False)
con.execute("LOAD SPATIAL;")

In [21]:
p1 = 'POINT (119.766904 -19.953996)'
p2 = 'POINT (119.766904 -20.953996)'

In [22]:
# Define the point in WKT format
p1 = 'POINT (119.766904 -19.953996)'

# Correct and format the query to compute the distance
q1 = f"""
    SELECT ST_Distance(
        ST_Transform(ST_Point('{}'), 'EPSG:4326', 'EPSG:3857'),
        ST_Transform(ST_GeomFromText('{p1}'), 'EPSG:4326', 'EPSG:3857')
    ) AS distance;
"""

# Execute the query
result = con.execute(q1).fetchdf()
print(result)


   distance
0       NaN


In [1]:
"""SELECT ST_AsText(
    ST_Transform(
        st_point(52.373123, 4.892360),
        'EPSG:4326',
        'EPSG:3857'
    )
);
"""



# ----
# POINT (544615.0239773799 6867874.103539125)

# -- Alternatively, let's say we got our input point from e.g., a GeoJSON file,
# -- which uses WGS84 but with [longitude, latitude] axis order. We can use the
# -- `always_xy` parameter to force the input geometry to be interpreted as having
# -- a [northing, easting] axis order instead, even though the source coordinate
# -- reference system definition (WGS84) says otherwise.

# SELECT ST_AsText(
#     ST_Transform(
#         -- note the axis order is reversed here
#         st_point(4.892360, 52.373123),
#         'EPSG:4326',
#         'EPSG:3857',
#         always_xy := true
#     )
# );
# ----
# POINT (544615.0239773799 6867874.103539125)

SyntaxError: unterminated string literal (detected at line 11) (2657946545.py, line 11)